In [10]:
import numpy as np

In [11]:
pi = RR.pi()

def integrate_curvature(kappa, s, srange=(0,1), theta_0=0, x_0=0, y_0=0, step=0.1):
    theta, x, y = var('θ, x, y')
    DE0 = kappa
    DE1 = cos(theta)
    DE2 = sin(theta)
    ICs = [srange[0], theta_0, x_0, y_0]

    P = desolve_system_rk4([DE0, DE1, DE2], [theta, x, y], ics=ICs, ivar=s, end_points=srange[1], step=step)
    return P


def splines_from_curvature(kappa, s, srange=(0,1), theta_0=0, x_0=0, y_0=0, step=0.1):
    P = integrate_curvature(kappa, s, srange, theta_0, x_0, y_0, step)

    x_spline = spline([(s, x) for s, theta, x, y in P])
    y_spline = spline([(s, y) for s, theta, x, y in P])

    return (x_spline, y_spline)


def spline_avg(f):
    a = f.list()[0][0]
    b = f.list()[-1][0]
    return f.definite_integral(a, b) / (b - a)


def splines_to_angular_momentum(x_0, y_0, x_1, y_1, dt, srange=(0,1), center=(0,0)):
    def theta(a, b): return arccos(a.inner_product(b) / (a.norm() * b.norm()))
    def v_x(z): return (x_0(z) - x_1(z)) / dt
    def v_y(z): return (y_0(z) - y_1(z)) / dt
    def v(z): return vector([v_x(z), v_y(z)])
    def r(z): return sqrt((x_0(z) - center[0])^2 + (y_0(z) - center[1])^2)
    def w(z): return v(z).norm() * sin(theta(v(z), vector([x_0(z) - center[0], y_0(z) - center[1]]))) # |v| * sin(theta)
    integrand = lambda z: w(z) * r(z) # w * r
    angular_momentum = numerical_integral(integrand, srange[0], srange[1])[0] # int_S w * r^2 ds
    return angular_momentum


def splines_to_moment(x, y, srange=(0,1), center=(0,0)):
    return numerical_integral(lambda z: ((x(z) - center[0])^2 + (y(z) - center[1])^2), srange[0], srange[1])[0]


def translate_spline(f, dy):
    return spline([(x, y + dy) for x, y in f.list()])


def rotate_splines(x, y, theta):
    x_list = x.list()
    y_list = y.list()
    
    R = matrix([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
    
    completed_x_list = [(s, x, y(s)) for s, x in x_list]
    completed_y_list = [(s, x(s), y) for s, y in y_list]
    
    rotated_x_spline = spline([(s, (R*vector([x, y]))[0]) for s, x, y in completed_x_list])
    rotated_y_spline = spline([(s, (R*vector([x, y]))[1]) for s, x, y in completed_y_list])
    
    return (rotated_x_spline, rotated_y_spline)


def splines_fix_center(x, y, center=(0,0)):
    x_bar = spline_avg(x)
    y_bar = spline_avg(y)
    return (translate_spline(x, -x_bar + center[0]), translate_spline(y, -y_bar + center[1]))

def splines_from_curvature_fix_center(kappa, s, srange=(0,1), theta_0=0, center=(0,0), step=0.1):
    x, y = splines_from_curvature(kappa, s, srange, theta_0, 0, 0, step)
    return splines_fix_center(x, y, center)


def flow_curvature(kappa, srange, arange, acount, theta_0=0, center=(0,0), step=0.1):
    splines = []
    rotated_splines = []
    space, dt = np.linspace(arange[0], arange[1], acount, retstep=True)
    total_counterrotation = 0
    curves = []
    for a in space:
        print(f"Calculating curve for a = {a}...")
        x, y = splines_from_curvature_fix_center(kappa(s, a), s, srange=srange, theta_0=theta_0, center=center, step=step)
        # print(x.list())
        # print(y.list())
        print("Done with curvature integration.")

        if len(splines) >= 1:
            print("Calculating angular momentum...")
            angular_momentum = splines_to_angular_momentum(splines[-1][0], splines[-1][1], x, y, dt, srange=srange, center=center)
            print("Done with angular momentum.")
            print("Calculating moment...")
            I = splines_to_moment(x, y, srange=srange, center=center)
            print("Done with moment.")
            print(f"I = {I}")
            angular_velocity = angular_momentum / I
            dtheta = angular_velocity * dt
            total_counterrotation += dtheta
            print(f"dtheta: {dtheta}\n")
            
        rotated_x, rotated_y = rotate_splines(x, y, total_counterrotation)
        rotated_translated_x, rotated_translated_y = splines_fix_center(rotated_x, rotated_y, center)
        rotated_splines.append((rotated_translated_x, rotated_translated_y))
        splines.append((x, y))
        
    return splines, rotated_splines

In [12]:
def metric_surf_rev_numerical(phi, psi):
    def g(u, v):
        X_u = vector([-phi(v)*sin(u), phi(v)*cos(u), 0])
        X_v = vector([phi.derivative(v)*cos(u), phi.derivative(v)*sin(u), psi.derivative(v)])
        E = X_u.dot_product(X_u)
        G = X_v.dot_product(X_v)
        return matrix([[E, 0], [0, G]])
    return g


def sample_metric_eigenvals(g, urange=(-1, 1), vrange=(-1, 1), ucount=5, vcount=5, sq_len=0.2):
    def angle(a, b):
        return arccos(a.inner_product(b) / (a.norm() * b.norm())) if a.norm() * b.norm() != 0 else 0
    
    squares = []
    transformed_squares = []
    ellipses = []

    uspace = np.linspace(urange[0], urange[1], ucount)
    vspace = np.linspace(vrange[0], vrange[1], vcount)
    space = cartesian_product_iterator([uspace, vspace])
    for u, v in space:
        g_curr = g(u, v)
        eigenvectors = g_curr.eigenvectors_right()
        eigvec = eigenvectors[0][1][0]
        k1 = eigenvectors[0][0]
        if eigenvectors[0][2] == 2:
            k2 = k1
        else:
            k2 = eigenvectors[1][0]
        
        square = matrix([[0, -sq_len/2, 0, sq_len/2], [-sq_len/2, 0, sq_len/2, 0]])
        cross = vector([eigvec[0], eigvec[1], 0]).cross_product(vector([1, 0, 0]))
        theta = arctan2(cross[2], eigvec.dot_product(vector([1, 0])))
        R = matrix([[cos(theta), sin(theta)], [-sin(theta), cos(theta)]])
        
        translate = matrix([[u, u, u, u], [v, v, v, v]])
        transformed = g_curr * R * square
        
        squares.append((R * square) + translate)
        transformed_squares.append(transformed + translate)
        ellipses.append((u, v, k1*sq_len/2, k2*sq_len/2, -theta))

    return squares, transformed_squares, ellipses

In [13]:
def curvature_surf_rev_numerical(phi):    
    def K(u, v):
        dphi = phi.derivative(v)
        d2phi = phi.derivative(v, order=2)
        dpsi = psi.derivative(v)
        d2psi = psi.derivative(v, order=2)
        
#         Go back to using formula with parametrization by arc length
        numerator = (-dpsi**2 * d2phi) + (dphi * dpsi * d2psi)
        denominator = phi(v) * (dphi**2 + dpsi**2)**2
        
        
        return numerator / denominator
    
    def k1(u, v):
        return 1 / phi(v)
    
    def k2(u, v):
        return K(u, v) / k1(u, v)
    
    return k1, k2, K

In [ ]:
phi(x) = 5 + sin(x)
psi(x) = cos(x)

space, step = np.linspace(0, 1, 100, retstep=True)
for t in space:
    g = metric_surf_rev_numerical(phi, psi)
    k1, k2, K = curvature_surf_rev_numerical(phi)
    def next_g(u, v):
        return g(u, v) - (K * g(u, v) * step)
#   Integrate next_g to get next_phi and next_psi
    